In [1]:
%load_ext autoreload
%autoreload 2

In [115]:
import os
import pormake as pm
from pathlib import Path
from tqdm import tqdm
from collections import defaultdict
import json
import pandas as pd
import numpy as np

In [141]:
from ase.neighborlist import natural_cutoffs
from ase import neighborlist

In [107]:
from pymatgen.io.ase import AseAtomsAdaptor

In [27]:
os.environ["CUDA_VISIBLE_DEVICES"]='0'

In [3]:
pm.log.disable_print()
pm.log.disable_file_print()   

>>> Console logs (under WARNING level) are disabled.


In [4]:
database = pm.Database(bb_dir=Path('./building_blocks/bbs/'))

# topology

In [5]:
topo_to_cn = {}
for topo_name in tqdm(database.topo_list):
    try:
        topo = database.get_topo(topo_name)
    except KeyboardInterrupt:
        break
    except Exception as e:
        continue
    cn = topo.unique_cn
    if len(cn) != 1:
        continue
    #print (topo, cn)
    topo_to_cn[topo_name] = cn[0]

  1%|█▊                                                                                                                                                     | 32/2596 [00:10<24:54,  1.72it/s]>>> Topology parsing fails: lcz
>>> Topology loading is failed: Invalid cgd file: lcz.
  3%|████▍                                                                                                                                                  | 76/2596 [00:28<08:26,  4.98it/s]>>> Topology parsing fails: tdz
>>> Topology loading is failed: Invalid cgd file: tdz.
  3%|████▍                                                                                                                                                  | 77/2596 [00:28<09:04,  4.62it/s]>>> Topology parsing fails: zim
>>> Topology loading is failed: Invalid cgd file: zim.
  3%|████▋                                                                                                                                                  | 80/2596 [00

In [6]:
topo_to_cn = {i:int(v) for i, v in topo_to_cn.items()}

In [7]:
with open('topo_to_cn.json', 'w') as f:
    json.dump(topo_to_cn, f)

# building_blocks

In [8]:
node = defaultdict(dict)
edge = defaultdict(dict)

for bb_name in database.bb_list:
    bb = database.get_bb(bb_name)
    cn_points = bb.n_connection_points
    if cn_points == 2:
        sd = edge
    else:
        sd = node
        
    if bb_name[0] == 'C':
        sd["center"][bb_name] = cn_points
    else:
        sd["linker"][bb_name] = cn_points

In [33]:
edge

defaultdict(dict,
            {'center': {'C44': 2,
              'C48': 2,
              'C41': 2,
              'C55': 2,
              'C61': 2,
              'C42': 2,
              'C46': 2,
              'C1': 2,
              'C33': 2,
              'C3': 2,
              'C63': 2,
              'C45': 2,
              'C59': 2,
              'C2': 2,
              'C4': 2,
              'C60': 2,
              'C64': 2,
              'C5': 2,
              'C51': 2,
              'C43': 2,
              'C47': 2},
             'linker': {'L38': 2,
              'L8': 2,
              'L4': 2,
              'L70': 2,
              'L31': 2,
              'L34': 2,
              'L18': 2,
              'L39': 2,
              'L1': 2,
              'L3': 2,
              'L23': 2,
              'L30': 2,
              'L25': 2,
              'L14': 2,
              'L22': 2,
              'L76': 2,
              'L2': 2,
              'L32': 2,
              'L43': 2,
           

# pre-calculated topo-bb

In [9]:
from itertools import chain

In [10]:
loc = pm.Locator()

In [11]:
candidate = defaultdict(list)

for topo_name, cn_topo in tqdm(topo_to_cn.items()):
    topo = database.get_topo(topo_name)
    
    for bb_name, cn_bb in chain(node['center'].items(), node['linker'].items()):
        if cn_topo != cn_bb:
            continue
        bb = database.get_bb(bb_name)
        rmsd = loc.calculate_rmsd(topo.unique_local_structures[0], bb)
        
        if rmsd < 0.3:
            candidate[topo_name].append(bb_name)

 29%|███████████████████████████████████████████                                                                                                            | 278/975 [00:28<00:42, 16.26it/s]/home/dudgns1675/library/PORMAKE/pormake/locator.py:20: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  U, rmsd = scipy.spatial.transform.Rotation.align_vectors(p, q)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 975/975 [01:37<00:00,  9.97it/s]


In [12]:
with open('topo_bb_candidate.json', 'w') as f:
    json.dump(candidate, f)

# All_Possible_list

In [13]:
from itertools import product

In [14]:
coflist = []

for topo_name, nd_list in candidate.items():
    for nd_name in nd_list:
        if nd_name[0] == 'C':
            eg_list = edge['linker']
        elif nd_name[0] == 'L':
            eg_list = edge['center']
        else:
            raise ValueError()
            
        for eg_name in eg_list:
            coflist.append(f'{topo_name}+{nd_name}+{eg_name}')

In [15]:
len(coflist)

153702

In [16]:
coflist

['ukx+C66+L38',
 'ukx+C66+L8',
 'ukx+C66+L4',
 'ukx+C66+L70',
 'ukx+C66+L31',
 'ukx+C66+L34',
 'ukx+C66+L18',
 'ukx+C66+L39',
 'ukx+C66+L1',
 'ukx+C66+L3',
 'ukx+C66+L23',
 'ukx+C66+L30',
 'ukx+C66+L25',
 'ukx+C66+L14',
 'ukx+C66+L22',
 'ukx+C66+L76',
 'ukx+C66+L2',
 'ukx+C66+L32',
 'ukx+C66+L43',
 'ukx+C66+L71',
 'ukx+C66+L36',
 'ukx+C66+L66',
 'ukx+C66+L72',
 'ukx+C66+L13',
 'ukx+C66+L37',
 'ukx+C66+L41',
 'ukx+C66+L15',
 'ukx+C66+L9',
 'ukx+C66+L40',
 'ukx+C66+L75',
 'ukx+C66+L74',
 'ukx+C66+L35',
 'ukx+C66+L33',
 'ukx+C66+L26',
 'ukx+C66+L42',
 'ukx+C66+L24',
 'ukx+C66+L69',
 'ukx+C66+L73',
 'ukx+C66+L68',
 'ukx+C66+L10',
 'ukx+C66+L44',
 'ukx+C69+L38',
 'ukx+C69+L8',
 'ukx+C69+L4',
 'ukx+C69+L70',
 'ukx+C69+L31',
 'ukx+C69+L34',
 'ukx+C69+L18',
 'ukx+C69+L39',
 'ukx+C69+L1',
 'ukx+C69+L3',
 'ukx+C69+L23',
 'ukx+C69+L30',
 'ukx+C69+L25',
 'ukx+C69+L14',
 'ukx+C69+L22',
 'ukx+C69+L76',
 'ukx+C69+L2',
 'ukx+C69+L32',
 'ukx+C69+L43',
 'ukx+C69+L71',
 'ukx+C69+L36',
 'ukx+C69+L66',
 'u

In [28]:
with open('coflist.txt', 'w') as f:
    for cof in coflist:
        f.write(cof+"\n")

In [19]:
builder = pm.Builder()

In [134]:
for name in tqdm(coflist):
    try:
        cof = build_from_seed(name, builder)
        
        if is_valid(cof):
            cof.write_cif(name)
        
    except KeyboardInterrupt:
        break
    except:
        continue

  0%|                                                                                                                                                              | 0/153702 [00:00<?, ?it/s]

ukx+C66+L38


  0%|                                                                                                                                                   | 1/153702 [00:01<83:13:36,  1.95s/it]

ukx+C66+L8


  0%|                                                                                                                                                   | 2/153702 [00:04<88:23:24,  2.07s/it]

ukx+C66+L4


  0%|                                                                                                                                                   | 3/153702 [00:06<89:32:01,  2.10s/it]

ukx+C66+L70


  0%|                                                                                                                                                   | 3/153702 [00:06<97:43:39,  2.29s/it]


# Build test

In [18]:
def build_from_seed(name:str, builder):
    print(name)
    tp, nd, eg = name.split('+')
    tp = database.get_topo(tp)
    nd = database.get_bb(nd)
    eg = database.get_bb(eg)
    
    current_node = {0:nd}
    current_edge = {}
    for i in tp.unique_edge_types:
        current_edge[tuple(i)] = eg
        
    current_mof = builder.build_by_type(tp, current_node, current_edge)
    return current_mof

In [132]:
def is_valid(cof):
    atoms = cof.atoms
    
    unique_atom, _ = get_unique_atoms(atoms)
    
    if len(unique_atom) > 300:
        return False
    
    elif max(atoms.cell.lengths()) > 90:
        return False
    
    return True

In [114]:
def get_unique_atoms(atoms):
    # get graph
    cutoff = natural_cutoffs(atoms)
    neighbor_list = neighborlist.NeighborList(cutoff, self_interaction=True, bothways=True)
    neighbor_list.update(atoms)
    matrix = neighbor_list.get_connectivity_matrix()

    # Get N, N^2
    numbers = atoms.numbers
    number_sqr = np.multiply(numbers, numbers)

    matrix_sqr = matrix.dot(matrix)
    matrix_cub = matrix_sqr.dot(matrix)
    matrix_sqr.data[:] = 1  # count 1 for atoms

    # calculate
    list_n = [numbers, number_sqr]
    list_m = [matrix, matrix_sqr, matrix_cub]

    arr = [numbers]

    for m in list_m:
        for n in list_n:
            arr.append(m.dot(n))

    arr = np.vstack(arr).transpose()

    uni, unique_idx, unique_count = np.unique(arr, axis=0, return_index=True, return_counts=True)

    # sort
    final_uni = uni[np.argsort(-unique_count)].tolist()
    final_unique_count = unique_count[np.argsort(-unique_count)].tolist()

    arr = arr.tolist()
    final_unique_idx = []
    for u in final_uni:
        final_unique_idx.append([i for i, a in enumerate(arr) if a == u])

    return final_unique_idx, final_unique_count

In [135]:
mof = build_from_seed(coflist[1], builder)

ukx+C66+L8


In [136]:
is_valid(mof)

False

In [137]:
i, v = get_unique_atoms(mof.atoms)
print (len(i))

43


In [140]:
mof.atoms.cell.lengths()

array([200.48612709, 199.19424172, 199.19423957])

In [113]:
mof.view()

In [110]:
st = AseAtomsAdaptor().get_structure(mof.atoms, )

In [112]:
st.get_primitive_structure()

Structure Summary
Lattice
    abc : 26.53769658953125 26.536851321240242 64.06341641901012
 angles : 89.99532945725015 90.00068075482871 119.9966966010974
 volume : 39072.19380479286
      A : 26.537696573603494 0.0009160847507624873 -7.849069771129865e-05
      B : -13.267893971604417 22.981894276216913 0.0017551813253616705
      C : -0.000571711051792797 0.0008072811645845524 64.06341641137271
PeriodicSite: C (7.2651, 6.4243, 1.7716) [0.4135, 0.2795, 0.0276]
PeriodicSite: C (6.9023, 7.4566, 2.6175) [0.4223, 0.3244, 0.0409]
PeriodicSite: O (5.6768, 8.0162, 2.2841) [0.3883, 0.3488, 0.0356]
PeriodicSite: B (5.3179, 7.2009, 1.1192) [0.3570, 0.3133, 0.0175]
PeriodicSite: O (6.3205, 6.1848, 0.7834) [0.3727, 0.2691, 0.0122]
PeriodicSite: C (7.7504, 7.8294, 3.6767) [0.4624, 0.3407, 0.0574]
PeriodicSite: C (8.9466, 7.1634, 3.8718) [0.4930, 0.3117, 0.0604]
PeriodicSite: O (9.6614, 7.6535, 4.9561) [0.5305, 0.3330, 0.0774]
PeriodicSite: B (8.7582, 8.7134, 5.4162) [0.5196, 0.3791, 0.0845]
Period

# test

In [107]:
current_node = {}
current_edge = {}

nd = database.get_bb('C34')
eg = database.get_bb('L18')

topo = test
print (topo.unique_cn, topo.unique_edge_types)

current_node[0] = nd

for i in topo.unique_edge_types:
    current_edge[tuple(i)] = eg

print (current_edge, current_node)
    
builder = pm.Builder()
current_cof = builder.build_by_type(topo, current_node, current_edge)

[3] [[0 0]]
{(0, 0): BuildingBlock: L18, # of connection points: 2} {0: BuildingBlock: C34, # of connection points: 3}


AttributeError: 'Atoms' object has no attribute '_pbc'

In [116]:
for tp in candidate.keys():
    #print (tp)
    if tp == 'hcb':
        print ('yes')
        try:
            atoms = database.get_topo(tp).atoms
        except Exception:
            raise Exception
            print (tp)
        break
    #break

yes


In [123]:
test = database.get_topo('bew').atoms

In [133]:
pm.Topology('./topologies/topo_2d/bew.cgd').atoms

Atoms(symbols='C3O6', pbc=True, cell=[[2.0, 0.0, -8.742277657347586e-08], [7.571031090947145e-08, 1.7320499420166016, -7.571031090947145e-08], [0.0, 0.0, 1.0]], tags=...)

In [132]:
database.get_topo('bew').atoms

Atoms(symbols='C3O6', pbc=True, cell=[[2.0, 0.0, -8.742277657347586e-08], [7.571031090947145e-08, 1.7320499420166016, -7.571031090947145e-08], [0.0, 0.0, 1.0]], tags=...)

In [124]:
test

AttributeError: 'Atoms' object has no attribute '_pbc'

In [105]:
test.atoms

AttributeError: 'Atoms' object has no attribute '_pbc'

In [102]:
'hcb' in candidate

True

In [95]:
topo.view()

AttributeError: 'Atoms' object has no attribute '_pbc'

In [65]:
current_cof.view()

In [66]:
c1.view()

In [32]:
loc = pm.Locator()
loc.calculate_rmsd(pcu.unique_local_structures[0], l1)

0.5898491875709481